# Setup IAM for Kinesis

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)


# Create Kinesis Role

In [ ]:
iam_kinesis_role_name = 'DSOAWS_Kinesis'

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesis.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "firehose.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesisanalytics.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }            
  ]
} 

In [ ]:
import json
import boto3
import time

from botocore.exceptions import ClientError

try:
    iam_role_kinesis = iam.create_role(
        RoleName=iam_kinesis_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Kinesis Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_kinesis = iam.get_role(RoleName=iam_kinesis_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(10)

In [ ]:
iam_role_kinesis_name = iam_role_kinesis['Role']['RoleName']
print('Role Name: {}'.format(iam_role_kinesis_name))

In [ ]:
iam_role_kinesis_arn = iam_role_kinesis['Role']['Arn']
print('Role ARN: {}'.format(iam_role_kinesis_arn))

In [ ]:
account_id = sts.get_caller_identity()['Account']

# Specify Stream Name

In [ ]:
stream_name = "dsoaws-data-stream"

# Specify Firehose Name

In [ ]:
firehose_name = "dsoaws-kinesis-data-firehose"

# Create Policy

In [ ]:
kinesis_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
         {
            "Effect": "Allow",
            "Action": [
                "kinesis:Get*",
                "kinesis:DescribeStream",
                "kinesis:Put*",
                "kinesis:List*",                
            ],
            "Resource": [
                "arn:aws:kinesis:{}:{}:stream/{}".format(region, account_id, stream_name)
            ]
         },
         {
            "Effect": "Allow",
            "Action": [
                "firehose:DeleteDeliveryStream",
                "firehose:PutRecord",
                "firehose:PutRecordBatch",
                "firehose:UpdateDestination",
            ],
            "Resource": [
                "arn:aws:firehose:{}:{}:deliverystream/{}".format(region, account_id, firehose_name)
            ]
         },
         {
            "Effect": "Allow",
            "Action": [
                "kinesisanalytics:*",
            ],
            "Resource": [
                "*"
            ]
         },
         {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/service-role/kinesis-analytics*"
         }        
    ]
}

print(kinesis_policy_doc)

# Update Policy

In [ ]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_kinesis_name,
    PolicyName='DSOAWS_KinesisPolicy',
    PolicyDocument=json.dumps(kinesis_policy_doc)
)

time.sleep(10)

In [ ]:
print(response)

In [ ]:
%store stream_name

In [ ]:
%store firehose_name

In [ ]:
%store iam_kinesis_role_name

In [ ]:
%store iam_role_kinesis_arn

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();